# 0N2_Animate_SatImagery

---
Read in and plot thermal infrared image(s) from MODIS and VIIRS level1b radiances. 

Overlay ERA5 atmospheric reanalysis surface atmospheric conditions (doi: 10.24381/cds.adbb2d47) and SIDEx buoy positions. 

### Import packages

In [1]:
%load_ext autoreload
%autoreload 2

from common_imports import *

from LIB_plot_sat import *
from LIB_plot_VIIRS import *
from LIB_plot_MODIS import *

import matplotlib.path as mpath
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from pyproj import Geod
g = Geod(ellps='WGS84')

# homemade functions
sys.path.append('./scripts/')
from LIB_scalebar import scale_bar
import matplotlib.ticker as ticker
import matplotlib.dates as mdates

## Import level1b MODIS files (HDF and GEO)
Download from: https://ladsweb.modaps.eosdis.nasa.gov/

In [2]:
# set paths for location of level1b hdf and geolocation files
# set provided folder type as path to folder and other as []
MainFolder = [] 
SingleFolder = []
#==============================================================
MainFolder = '/Users/mackenziejewell/Desktop/filenames/'
# SingleFolder = '/Users/mackenziejewell/Desktop/temp/VIIRS/'
# SingleFolder = '/Users/mackenziejewell/Desktop/filenames/plot/2021085/'
#==============================================================


#==============================================================
sensor = 'VIIRS'
# sensor = 'MODIS'
#==============================================================

if str(sensor) == 'VIIRS':
    satellite_labels = [('VNP03MOD','VNP02MOD'), ('VJ103MOD','VJ102MOD')]
elif str(sensor) == 'MODIS':
    satellite_labels = [('MOD03','MOD021KM'), ('MYD03','MYD021KM')]

Image_Meta_paired = pair_images_meta(MainFolder = MainFolder, SingleFolder = SingleFolder, 
                                     sensor = sensor, satellite_labels = satellite_labels,
                                     min_geofile_sizeMB = 28, min_imfile_sizeMB = 50, max_diff_minutes = 20)
    

Search within main folder: /Users/mackenziejewell/Desktop/filenames/
Pair 0
------
2021-03-01 12:00:00
2021-03-01 12:06:00

Pair 1
------
2021-03-01 15:18:00
2021-03-01 15:24:00

Pair 2
------
2021-03-01 20:18:00
2021-03-01 20:24:00

Pair 3
------
2021-03-01 22:54:00
2021-03-01 23:00:00

Pair 4
------
2021-03-02 12:30:00
2021-03-02 12:36:00

Pair 5
------
2021-03-02 15:54:00

Pair 6
------
2021-03-02 20:00:00
2021-03-02 20:06:00

Pair 7
------
2021-03-02 22:30:00
2021-03-02 22:36:00

Pair 8
------
2021-03-03 12:12:00
2021-03-03 12:18:00

Pair 9
------
2021-03-03 15:30:00
2021-03-03 15:36:00

Pair 10
------
2021-03-03 20:30:00
2021-03-03 20:36:00

Pair 11
------
2021-03-03 23:06:00
2021-03-03 23:12:00

Pair 12
------
2021-03-04 12:42:00
2021-03-04 12:48:00

Pair 13
------
2021-03-04 15:12:00
2021-03-04 15:18:00

Pair 14
------
2021-03-04 20:12:00
2021-03-04 20:18:00

Pair 15
------
2021-03-04 22:48:00
2021-03-04 22:54:00

Pair 16
------
2021-03-05 12:24:00
2021-03-05 12:30:00

Pair 17
-

## Specify which pairs of images to plot

In [3]:
# specify which pairs of images to plot
# either 'All' or index(es) of images to plot (e.g. [1,3,6])
#==============================================================
# RunPair = [0,240]
RunPair = np.arange(0,120) 
# RunPair = 'All'
#==============================================================

## ECMWF data

In [4]:
# name and directory for ECMWF atmospheric data (nc file type)
# or set = None if you don't want to include
#==============================================================
# ECMWF = None
ECMWF = '/Volumes/Jewell_EasyStore/ECMWF/annual/hourly/ERA5_2021.nc'
#==============================================================


## Buoy coordinate data

In [5]:
# set list of path+name of csv files containing coordinates
# or set = None if not adding buoy coordinates
#==============================================================
# csv_directory = None
buoy_file = './data/BuoyCoordinates_cln_v0.nc'
#==============================================================
ds = xr.open_dataset(buoy_file)
ds.close()
buoy_time = pd.to_datetime(ds.time.values)


## Specify other plot parametres and layers

In [6]:
# PLOT PARAMETERS

# set plot range and map_projection
#==============================================================
lat_range = [69.5, 72.25]
lon_range = [198, 232]
extent = [lon_range[0], lon_range[1], lat_range[0], lat_range[1]]
map_projection = ccrs.NorthPolarStereo(central_longitude=205)
#==============================================================

# specify band of data to plot
#==============================================================
if str(sensor) == 'MODIS':
    band = '31'   # Thermal MODIS: Infrared (TIR) at 10.780–11.280 micrometers
elif str(sensor) == 'VIIRS':
    band = 'M15'  # Thermal VIIRS: longwave IR 10.26 - 11.26 micrometers
#==============================================================


# set colorscale for image
#==============================================================
ice_cmap = mpl.cm.Blues
# ice_cmap = cmocean.cm.ice_r
#==============================================================


# whether or not to suppress prints
#==============================================================
quiet = True
#==============================================================

# hide known warnings that result in many printed warning statements
#==============================================================
# ignore shapely warning for geographic plots
import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
# pcolormesh shading warning
warnings.filterwarnings("ignore", module = "matplotlib\..*" )
warnings.filterwarnings("ignore", module = "cartopy\..*" )
#==============================================================



def flatten_pair_data(lat, lon, _image_):
    all_lats = np.array([])
    all_lons = np.array([])
    all_rad = np.array([])
    all_lats = np.append(all_lats, lat[0].flatten())
    all_lons = np.append(all_lons, lon[0].flatten())
    all_rad = np.append(all_rad, _image_[0].flatten())
    if len(lat) == 2:
        all_lats = np.append(all_lats, lat[1].flatten())
        all_lons = np.append(all_lons, lon[1].flatten())
        all_rad = np.append(all_rad, _image_[1].flatten())
    return all_lats, all_lons, all_rad


# grab Alaska and Canada coast line strings
AK = list(cfeat.COASTLINE.with_scale('10m').geometries())[485]
CD = list(cfeat.COASTLINE.with_scale('10m').geometries())[481]

# grab coords
AKlats = AK.coords.xy[1]
AKlons = AK.coords.xy[0]
CDlats = CD.coords.xy[1]
CDlons = CD.coords.xy[0]


# background poly (MUCH faster than using cfeat.OCEAN)
lon_bg = np.array([-165,-165,-120,-120])
lat_bg = np.array([69,74,72,65])
bg_poly = make_polygon(np.stack((lon_bg[::-1], lat_bg[::-1]),axis=1), ellipsoid='WGS84', quiet=True)

## make plot

In [7]:


FS = 8




starttime = datetime.now()

# RUN TO MAKE PLOT, ADJUST APPEARANCE OF LAYERS BELOW AS NEEDED
#==============================================================

# Run through all pairs of images given in RunPair and make plots
#----------------------------------------------------------------
if str(RunPair) == 'All':
    RunPair = np.arange(0,np.max(Image_Meta_paired[:,4])+1)
for ii in RunPair:
    # grab metadata from current_set of paired images
    #------------------------------------------------
    # grab current_set of paired images to run through
    current_set = Image_Meta_paired[np.where(Image_Meta_paired[:,4]==ii)[0]]
    # start empty lists to fill with image names, paths, and dates
    IMG_filename=[]
    GEO_filename=[]
    # add data from all images in current_set to above lists
    counter = 0
    for image_meta in current_set:
        # grab date and ImageName for saving from first file in current_set
        if counter == 0:
            ImageDate = image_meta[0]
            ImageName = image_meta[3]+image_meta[2][0:22]
        IMG_filename = np.append(IMG_filename, image_meta[3]+image_meta[2])
        GEO_filename = np.append(GEO_filename, image_meta[3]+image_meta[1])
        counter+=1
    

    if not quiet:
        print('Save with name {}'.format(ImageName))    
        print('Image is from: {} UTC (day {} of {})'.format(ImageDate,ImageDate.strftime('%j'), ImageDate.strftime('%Y')))
    # grab data from current_set of paired images
    #--------------------------------------------
    # start empty lists to fill with imagery data and coordinates
    _image_ = []
    lat = []
    lon = []

    # for all images in current_set
    for jj in range(len(current_set)):

        # import imagery data, mask invalid values
        # and import geo data
        #-----------------------------------------

        if str(sensor) == 'MODIS':
            _level1bimage_ = load_MODISband(IMG_filename[jj], 'EV_1KM_Emissive', band, 'radiance')
            LAT, LON = get_MODISgeo(GEO_filename[jj])

        elif str(sensor) == 'VIIRS':
            _level1bimage_ = load_VIIRS_band(IMG_filename[jj], band = 'M15')
            LAT, LON = get_VIIRS_geo(GEO_filename[jj])


        # add imagery and coordinates for this file to the lists
        #-------------------------------------------------------
        _image_.append(_level1bimage_)
        lat.append(LAT)
        lon.append(LON)

    
    nearest_hour_date = ImageDate.replace(second=0, microsecond=0, minute=0, hour=ImageDate.hour) + timedelta(hours=ImageDate.minute//30)
    
    #==============
    # PLOT IMAGERY
    #==============
    # create figure
    #--------------
    # dynamic color scale
    all_lats, all_lons, all_rad = flatten_pair_data(lat, lon, _image_)
    all_rad[all_rad.data == 65533] = np.nan

    cond = (all_lats > 70).astype(int)+(all_lats < 77).astype(int)+(all_lons > 200).astype(int)+(all_lons < 240).astype(int)
    min_val = np.nanpercentile(all_rad.data[cond == 4], 1)-0.1
    max_val = np.nanpercentile(all_rad.data[cond == 4], 99)+0.75
    cscale = [min_val, max_val]
    divnorm=matplotlib.colors.TwoSlopeNorm(vmin=min_val, vcenter=min_val+0.7*(max_val-min_val), vmax=max_val)

    sp = 5
    fig, ax = plt.subplots(subplot_kw=dict(projection=map_projection), figsize=(3.2,3), facecolor='white')
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    ax.add_geometries([bg_poly], facecolor=ice_cmap(0.3), edgecolor='k', crs=ccrs.Geodetic(), zorder=-1)

    for ii in range(0,len(_image_)):
        ax.pcolormesh(lon[ii], lat[ii], _image_[ii],norm = divnorm,
                      cmap=ice_cmap, shading='nearest', zorder=1, transform=ccrs.PlateCarree())

    #======================
    # ADD CARTOPY FEATURES
    #======================
    add_land(ax,  scale='10m', color=[0.85,0.85,0.85], alpha=1, fill_dateline_gap=True, zorder=3)
    add_coast(ax, scale='10m', color=[0.7,0.7,0.7], linewidth=0.5, alpha=1, zorder=4)
#     add_grid(ax, lats=np.arange(70,90,5), lons=np.arange(100,300,20), linewidth=1, color='gray', alpha=0.3, zorder=10)


    # label date
    #======================
    add_date(fig, ax, nearest_hour_date, date_format='%Y-%m-%d %H:00',
             boxstyle='round,pad=0.2,rounding_size=0.2',
             method='manual', facecolor='white', edgecolor='None',
             x = 0.185, y = 4.05, textcolor='k', fontsize=FS+5, zorder=10)

    # import era5
    #=============
    # grab nearest date index from ECMWF file
    if not quiet:
        print('Add wind data: nearest date ECMWF --> {}'.format(nearest_hour_date))
    ds_era = xr.open_dataset(ECMWF).sel(time=nearest_hour_date)
    ds_era.close()
    u10  = ds_era.u10.values
    v10  = ds_era.v10.values
    msl  = ds_era.msl.values/100
    Lons, Lats = np.meshgrid(ds_era.longitude.values, ds_era.latitude.values)


    # wind vectors
    #======================
#     wind_vec = ax.quiver(Lons, Lats, *fix_cartopy_vectors(u10, v10, Lats),
#                         regrid_shape=7, scale=150, width=0.003, color=[0.6,0.6,0.6], headwidth=4, pivot='mid',
#                         headaxislength=4, headlength=5, transform=ccrs.PlateCarree(), zorder=8)
#     qk = ax.quiverkey(wind_vec, 0.44, 1.02, 10,  '10m wind\n(10 m s$^{-1}$)',
#                       labelpos='N', coordinates='axes', fontproperties={'size':FS})

    # grab buoy data
    #======================
    ds3 = xr.open_dataset('./data/buoy_wind_MarchApril2021.nc')
    ds3.close()
    
    range_ds = ds3.sel(time=slice(ds3.time[0], nearest_hour_date))

    curr_ds = ds3.sel(time=nearest_hour_date)
    curr_lats = curr_ds.latitude.values
    curr_lons = curr_ds.longitude.values
    curr_ice_u = curr_ds.ice_u.values
    curr_ice_v = curr_ds.ice_v.values
    curr_ice_speed = np.sqrt(curr_ice_u**2 + curr_ice_v**2)
    curr_wind_speed = np.sqrt(curr_ds.wind_u.values**2 + curr_ds.wind_v.values**2)
    
    buoys = {}
    buoys['lat'] = curr_lats[np.isnan(curr_lats)==False]
    buoys['lon'] = curr_lons[np.isnan(curr_lats)==False]
    buoys['ice_u'] = curr_ice_u[np.isnan(curr_lats)==False]
    buoys['ice_v'] = curr_ice_v[np.isnan(curr_lats)==False]
    buoys['sp_ratio'] = curr_ice_speed[np.isnan(curr_lats)==False]/curr_wind_speed[np.isnan(curr_lats)==False]
    
    # pick center coordinates for lagrangian inset
    # if buoy 50 not yet deployed, use buoy 23
    if np.isnan(curr_ds.sel(buoyID='50').latitude.values):
        center_lat = curr_ds.sel(buoyID='23').latitude.values-0.0225
        center_lon = curr_ds.sel(buoyID='23').longitude.values-0.5
    # once buoy 50 deployed, track buoy 50 (right next to 23, but better shows shearing events on contact w/ coast)
    else:
        first_nonnan = np.where(np.isnan(ds3.sel(buoyID='50').latitude)==False)[0][0]
        lat_diff = (ds3.sel(buoyID='23').latitude[first_nonnan].values-ds3.sel(buoyID='50').latitude[first_nonnan].values)
        lon_diff = (ds3.sel(buoyID='23').longitude[first_nonnan].values-ds3.sel(buoyID='50').longitude[first_nonnan].values)
        center_lat = curr_ds.sel(buoyID='50').latitude.values-0.0225+lat_diff
        center_lon = curr_ds.sel(buoyID='50').longitude.values-0.5+lon_diff
        
    buoy_cmap = cmocean.cm.haline_r


    # inset map 1
    #======================
    #======================
    #======================
    
    # create inset plots
    #-------------------
    size = 2.45
    axins = inset_axes(ax, width="100%", height="100%", loc='upper left',
                       bbox_to_anchor=(-0.74,1.25,size,size), bbox_transform=ax.transAxes, 
                       axes_class=cartopy.mpl.geoaxes.GeoAxes, 
                       axes_kwargs=dict(map_projection=map_projection))
    # buoys
    #======================
    
    
    # find lon, lat 150 km in either direction
    distance = 145000 # m
    endlon, endlat, backaz = g.fwd(center_lon, center_lat, 90, distance)
    buffer_lon = np.abs(center_lon-endlon)
    endlon, endlat, backaz = g.fwd(center_lon, center_lat, 0, distance)
    buffer_lat = np.abs(center_lat-endlat)
    

    extent_mini = [center_lon-buffer_lon, center_lon+buffer_lon,
                   center_lat-buffer_lat, center_lat+buffer_lat]
    axins.set_extent(extent_mini, ccrs.PlateCarree())

    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    axins.set_boundary(circle, transform=axins.transAxes)
    out = verts * radius + center
#     axins.plot(out[:,0], out[:,1], c='k', lw=1, transform=axins.transAxes, zorder=10)
    
    
    #======================
    # grab coordinates of circle boundary
    lon_circ = np.array([])
    lat_circ = np.array([])
    proj_cart = ccrs.PlateCarree()
    for ii in range(len(out)):
        p_a = out[ii,:]
        # convert from Axes coordinates to display coordinates
        p_a_disp = axins.transAxes.transform(p_a)
        # convert from display coordinates to data coordinates
        p_a_data = axins.transData.inverted().transform(p_a_disp)
        # convert from data to cartesian coordinates
        p_a_cart = proj_cart.transform_point(*p_a_data, src_crs=map_projection)
        lon_circ = np.append(lon_circ, p_a_cart[0])
        lat_circ = np.append(lat_circ, p_a_cart[-1])
    
    # find portion of Alaska coast within zoom region, add to plot
    # find AK coods within circle polygon
    circle_poly = Polygon(list(zip(lon_circ[::-1], lat_circ[::-1])))
    AKlats_within = np.array([])
    AKlons_within = np.array([])
    for ii in range(len(AKlats)):
        point = Point(AKlons[ii], AKlats[ii])
        if circle_poly.contains(point):
            AKlats_within = np.append(AKlats_within, AKlats[ii])
            AKlons_within = np.append(AKlons_within, AKlons[ii])
    #======================    
        
    
    
#     add_land(axins,  scale='50m', color=[0.85,0.85,0.85], alpha=1, fill_dateline_gap=True, zorder=3)
    
    # plot buoys
    #======================
    # large map axis
    ice_vec2 = ax.quiver(buoys['lon'], buoys['lat'], *fix_cartopy_vectors(buoys['ice_u'], buoys['ice_v'], buoys['lat']),
                        scale=640, width=0.0025, headwidth=5, headaxislength=5, headlength=5, transform=ccrs.PlateCarree(), zorder=8)
    buoyc = ax.scatter(buoys['lon'], buoys['lat'], s=4,
               c=buoys['sp_ratio'], cmap=buoy_cmap, vmin=0, vmax=3,
               edgecolor='k', lw=0.25, transform=ccrs.PlateCarree(), zorder=9)
    
    # inset map axis
    ice_vec = axins.quiver(buoys['lon'], buoys['lat'], *fix_cartopy_vectors(buoys['ice_u'], buoys['ice_v'], buoys['lat']),
              scale=160, width=0.004, headwidth=5, headaxislength=5, headlength=5, transform=ccrs.PlateCarree(), zorder=8)
    
    qk = axins.quiverkey(ice_vec, 0.95, 0.865, 20,  'SIDEx buoy\n(20 cm s$^{-1}$)',
                      labelpos='N', coordinates='axes', fontproperties={'size':FS})

    
#     ax.quiverkey(ice_vec2, 0.95, 2.865, 80,  'SIDEx buoy\n(20 cm s$^{-1}$)',
#                  labelpos='N', coordinates='axes', 
#                  fontproperties={'size':FS})
    
    axins.scatter(buoys['lon'], buoys['lat'], s=5,
                  c=buoys['sp_ratio'], cmap=buoy_cmap, vmin=0, vmax=3,
                  edgecolor='k', lw=0.4, transform=ccrs.PlateCarree(), zorder=9)
    add_colorbar(fig, ax, [buoyc], cb_placement='top', cb_orientation='horizontal', 
                 cb_width=0.0225, cb_length_fraction=[0.9, 1.045], cb_pad=0.09, cb_ticks=[0,3], 
                 cb_ticklabels=['0%','3%'], cb_extend='neither', cb_label='', labelpad='auto', 
                 tick_kwargs = {'pad':2, 'length':0.2, 'labelsize':FS},
                 cb_label_placement='auto', cb_tick_placement='bottom', cb_labelsize=FS)
    ax.text(0.98, 1.43, 'Speed ratio', c='k', horizontalalignment='center', verticalalignment='center', size=FS, clip_on=False, transform=ax.transAxes, zorder=10)
    
#     add_colorbar(fig, ax, [buoyc], cb_placement='top', cb_orientation='vertical', 
#                  cb_width=0.09, cb_length_fraction=[0.95, 0.975], cb_pad=0.055, cb_ticks=[0,3], 
#                  cb_ticklabels=['0%','3%'], cb_extend='neither', cb_label='', labelpad='auto',
#                  cb_label_placement='auto', cb_tick_placement='auto', cb_labelsize=FS, 
#                  draw_edges=False, edge_params=['k', 2])
#     ax.text(0.85, 1.3, 'Speed\nratio', c='k', horizontalalignment='center', verticalalignment='center', size=FS, clip_on=False, transform=ax.transAxes, zorder=10)
    
    
    # imagery
    #========
    cond = (all_lats > center_lat-buffer_lat).astype(int)+(all_lats < center_lat+buffer_lat).astype(int)+(all_lons > center_lon-buffer_lon+360).astype(int)+(all_lons < center_lon+buffer_lon+360).astype(int)
    min_val = np.nanpercentile(all_rad.data[cond == 4], 1)-0.1
    max_val = np.nanpercentile(all_rad.data[cond == 4], 99)+0.75
    cscale = [min_val, max_val]
    divnorm=matplotlib.colors.TwoSlopeNorm(vmin=min_val, vcenter=min_val+0.7*(max_val-min_val), vmax=max_val)
    for ii in range(0,len(_image_)):
        axins.pcolormesh(lon[ii], lat[ii], _image_[ii],norm = divnorm,
                         cmap=ice_cmap, shading='nearest', zorder=1, transform=ccrs.PlateCarree(), 
                         clip_path=(circle, axins.transAxes))

    
    # plot coast on general map

    # plot land on inset map
    NAM = list(cfeat.LAND.with_scale('10m').geometries())[0][1]
    poly_within = circle_poly.intersection(NAM)
    if str(poly_within.type) == 'Polygon':
        px = poly_within.exterior
        if px.is_ccw==False:
            px.coords = list(px.coords)[::-1]
        axins.add_geometries([px], facecolor=[0.85,0.85,0.85], edgecolor='None', crs=ccrs.Geodetic(), zorder=3)
    elif str(poly_within.type) == 'MultiPolygon':
        for poly in poly_within:
            px = poly.exterior
            if px.is_ccw==False:
                px.coords = list(px.coords)[::-1]
            axins.add_geometries([px], facecolor=[0.85,0.85,0.85], edgecolor='None', crs=ccrs.Geodetic(), zorder=3)

    # plot coast on inset map
    axins.plot(AKlons_within, AKlats_within, transform=ccrs.PlateCarree(), lw=0.5, c=[0.7,0.7,0.7], zorder=4)
#     ax.text(-0.25, 0.425, '? km radius', horizontalalignment='center', size=FS, clip_on=False, transform=ax.transAxes, zorder=10)
    
    # add circle boundary to both plots
    ax.add_geometries([circle_poly], facecolor='None', edgecolor='k', crs=ccrs.Geodetic(), zorder=100)
    axins.add_geometries([circle_poly], facecolor='None', edgecolor='k', crs=ccrs.Geodetic(), zorder=100)
    
    
    
    
    scale_bar(axins, (0.045, 0.02), 50, metres_per_unit=1000, vert_length = 0.01, unit_name='km', color='black', 
          linewidth=1, text_offset=0.015, text_kwargs = {'fontsize': FS-1})
    
    scale_bar(ax, (0.07, 0.05), 200, metres_per_unit=1000, vert_length = 0.025, unit_name='km', color='black', 
          linewidth=1, text_offset=0.04, text_kwargs = {'fontsize': FS-1}, plot_kwargs={'zorder' : 101})
    
    # plot overall buoy track
    ax.plot(range_ds.sel(buoyID='23').longitude.values, 
            range_ds.sel(buoyID='23').latitude.values, c='k', linestyle=(0,(1,1)), linewidth = 0.75, zorder = 7, transform=ccrs.PlateCarree())
    
    
    # data subplots 
    #==========================
    width = 1
    height = 0.7
    x1 = 1.15
    

    # cumulative displacement
    #==========================
    ax_5 = inset_axes(ax, width="100%", height="100%", loc='upper left',
                      bbox_to_anchor=(x1,3.2,width,height), bbox_transform=ax.transAxes)
    
    # grab displacements traveled by central buoy
    dx = range_ds.ice_dx/100000 # cm to km
    dy = range_ds.ice_dy/100000 # cm to km

    beta = (90-(-67.5))*units('degree') # change from azimuth to CCW from east
    # calculate zonal, meridional displacements for azimuth
    DX = (np.cos(beta.to('radian')))
    DY = (np.sin(beta.to('radian')))

    # find local components along this direction
    compMs = (((dx*DX) + (dy*DY))/(np.sqrt(DX**2 + DY**2)))

    for ID in ds.buoyID:
        ax_5.plot(range_ds.time, np.cumsum(compMs.sel(buoyID = ID)), c='lightgray', lw=1, zorder=0)
        
    ax_5.plot(range_ds.time, np.cumsum(compMs.sel(buoyID = '23')), c='k', lw=1, zorder=2)
    ax_5.yaxis.set_major_locator(ticker.MultipleLocator(200))
    ax_5.yaxis.set_minor_locator(ticker.MultipleLocator(50))
#     ax_5.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.0f} km"))
    
#     ax_5.set_yticklabels(['0 km','200 km','400 km'])
    ax_5.set_ylim(0,400)
    ax_5.set_ylabel('WNW\ntransport\n(km)', rotation=0, fontsize=FS, labelpad=23, va='center', y=0.8)
    
    # label lines
    hy = 1.2
    ax_5.plot([0,0.1], [hy, hy],  c='lightgray', lw=1, clip_on=False, transform=ax_5.transAxes, zorder=10)
    ax_5.plot([0.4,0.5], [hy, hy],  c='k', lw=1, clip_on=False, transform=ax_5.transAxes, zorder=10)
    
    ax_5.text(0.125, hy, 'all buoys', c='k', horizontalalignment='left', verticalalignment='center', size=FS, clip_on=False, transform=ax_5.transAxes, zorder=10)
    ax_5.text(0.525, hy, 'central buoy', c='k', horizontalalignment='left', verticalalignment='center', size=FS, clip_on=False, transform=ax_5.transAxes, zorder=10)
    
    
    
    # wind-ice angle deflection
    #==========================
    ax_2 = inset_axes(ax, width="100%", height="100%", loc='upper left',
                      bbox_to_anchor=(x1,2.2,width,height), bbox_transform=ax.transAxes)
    
    for ID in ds.buoyID:
        ax_2.plot(range_ds.time, range_ds.sel(buoyID = ID).deflec_angle, c='lightgray', lw=1, zorder=0)
    ax_2.plot(range_ds.time, range_ds.deflec_angle.sel(buoyID='23'), c='k', lw=1, zorder=2)

    ax_2.hlines(0, datetime(2021, 3, 1, 0, 0), datetime(2021, 5, 1, 0, 0), colors='k', linewidths=0.5, zorder=-1)
    ax_2.yaxis.set_major_locator(ticker.MultipleLocator(180))
    ax_2.yaxis.set_minor_locator(ticker.MultipleLocator(30))
    ax_2.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.0f}°"))
    ax_2.set_ylim(-190,190)
    ax_2.set_ylabel('Wind-ice\ndeflection\nangle', rotation=0, fontsize=FS, labelpad=17, va='center', y=0.8)
    
    
    # wind-ice speed ratio
    #==========================
    ax_3 = inset_axes(ax, width="100%", height="100%", loc='upper left',
                      bbox_to_anchor=(x1,1.2,width,height), bbox_transform=ax.transAxes)
    ice_speed = np.sqrt(range_ds.ice_u**2 + range_ds.ice_v**2)
    wind_speed = np.sqrt(range_ds.wind_u**2 + range_ds.wind_v**2)
    sp_ratio = ice_speed/wind_speed

    for ID in ds.buoyID:
        ax_3.plot(range_ds.time, sp_ratio.sel(buoyID = ID), c='lightgray', lw=1, zorder=0)
    ax_3.plot(range_ds.time, sp_ratio.sel(buoyID='23'), c='k', lw=1, zorder=2)
#     ax_3.plot(range_ds.time, sp_ratio.median(dim='buoyID'), c='k', lw=1)
    ax_3.yaxis.set_major_locator(ticker.MultipleLocator(5))
    ax_3.yaxis.set_minor_locator(ticker.MultipleLocator(1))
    ax_3.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.0f}%"))
    ax_3.set_ylim(0,10)
    ax_3.set_ylabel('Buoy : wind\nspeed ratio', rotation=0, fontsize=FS, labelpad=23, va='center', y=0.8)
    
    
    # SBS open area
    #==========================
    ax_4 = inset_axes(ax, width="100%", height="100%", loc='upper left',
                      bbox_to_anchor=(x1,0.2,width,height), bbox_transform=ax.transAxes)
    
    ds_area = xr.open_dataset('./data/SBS_open_area.nc')
    ds_area.close()
    ds_crop = ds_area.sel(time=slice(datetime(2021, 3, 1, 1, 0), nearest_hour_date))

    ax_4.bar(pd.to_datetime(ds_crop.time.values)+timedelta(hours=12), ds_crop.N_oa_g20.values/1000,
             width = timedelta(days=1), facecolor='k', zorder=0)

    ax_4.yaxis.set_major_locator(ticker.MultipleLocator(10))
    ax_4.yaxis.set_minor_locator(ticker.MultipleLocator(5))
    ax_4.set_ylim(0,30)
    ax_4.set_ylabel('SBS\nopen area\n($\mathrm{10^3\;km^2}$)', rotation=0, fontsize=FS, labelpad=27, va='center', y=0.8)
    
    
    
    for axis in [ax_2, ax_3, ax_4, ax_5]:
        axis.yaxis.set_label_position("right")
        axis.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
        axis.set_xticks([datetime(2021, 3, 1), datetime(2021, 3, 15), datetime(2021, 4, 1), datetime(2021, 4, 15), datetime(2021, 5, 1)])
        axis.set_xlim(datetime(2021, 3, 1, 0, 0), datetime(2021, 5, 1, 0, 0))
        axis.tick_params(axis='both', which='major', labelsize=FS)
        axis.tick_params(axis='both', which='minor', labelsize=FS)
        axis.yaxis.tick_right()
        axis.vlines(nearest_hour_date, *axis.get_ylim(), colors='lightcoral', zorder=1)
    
    for axis in [ax_2, ax_3,ax_5]:
        axis.set_xticklabels([])
    ax_4.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    ax_4.set_xlabel('Date in 2021', fontsize=FS)
    

#     ax.text(0.05, 0.03, 'Thermal infrared VIIRS, MODIS imagery', c=[0.7,0.7,0.7], horizontalalignment='left', verticalalignment='center', size=FS-1.5, clip_on=False, transform=ax.transAxes, zorder=10)
    ax.text(0.975, -0.1, 'Animation by MacKenzie Jewell', c=[0.7,0.7,0.7], horizontalalignment='right', verticalalignment='top', size=FS-1, clip_on=False, transform=ax.transAxes, zorder=10)
#     ax.text(0.25, 0.15, '$\it{Alaska}$', c=[0.7,0.7,0.7], horizontalalignment='left', verticalalignment='top', size=FS, clip_on=False, transform=ax.transAxes, zorder=10)
    
    fig.savefig(ImageName+'_v2.png',bbox_inches="tight", pad_inches = 0.35, dpi=300)
    fig.clear()
    plt.close(fig) 

    

print('\n\n\n')
print(f'>>> runtime: {datetime.now()-starttime}')






>>> runtime: 1:50:57.480616
